In [1]:
# IMPORTING THE DATA

import numpy as np
import csv

# set up lists of points
metrics: list[list[float]] = [[], [], []]
color_scores: list[list[float]] = [[], [], []]
depth_scores: list[list[float]] = [[], [], []]

# imports a *.txt file into metrics, color_scores and depth_scores
def import_file(name: str) -> None:
    global metrics
    global color_scores
    global depth_scores

    # read all text
    file = open(name)
    txt: str = file.read()

    # parse into csv
    lines = txt.splitlines()
    reader = csv.reader(lines, delimiter=',')

    # put into array
    data = np.array([row for row in reader if row[0].isdigit()])

    # get the metrics of this file
    file_metrics = [float(v) / 10.0 for v in data[:, 0]]
    file_color_scores = [float(v) for v in data[:, 1]]
    file_depth_scores = [float(v) for v in data[:, 2]]

    # split by source image, put in lists
    for i in range(3):
        metrics[i] += file_metrics[(11 * i):(11 * i + 11)]
        color_scores[i] += file_color_scores[(11 * i):(11 * i + 11)]
        depth_scores[i] += file_depth_scores[(11 * i):(11 * i + 11)]

# import all text files
for i in range(5):
    import_file(f"data/data_{i}.txt")

# map the two values into a single range
mapped_scores = [[(d / (d + c) if (d + c != 0) else 0.5)
    for (c, d) in zip(color_scores[i], depth_scores[i])] for i in range(3)]

# flatten all lists
all_mapped_scores = sum(mapped_scores, [])
all_metrics = sum(metrics, [])
all_color_scores = sum(color_scores, [])
all_depth_scores = sum(depth_scores, [])


In [ ]:
# LINEAR REGRESSION OF SCORES (3D)

import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# get x and y to regress
x = list(map(list, zip(all_color_scores, all_depth_scores)))
y = all_metrics

# linear regression
reg = LinearRegression().fit(x, y)

[m1, m2], b = reg.coef_, reg.intercept_

print(f"metric = {m1 : .3f} * color_score + {m2 : .3f} * depth_score +{b : .3f}")
print("R^2 =", reg.score(x, y))

# create plot
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

# scatter points, colored by source image
for (i, cc) in zip(range(3), ['r', 'g', 'b']):
    ax.scatter(color_scores[i], depth_scores[i], metrics[i], color=cc)

# plot regression plane
xx, yy = np.meshgrid(range(11), range(11))
np_xx, np_yy = np.array(xx) * 0.1, np.array(yy) * 0.1

z = m1 * np_xx + m2 * np_yy + b

ax.plot_surface(np_xx, np_yy, z, alpha=0.2)

# set axes
ax.set_xlabel(('$s_{C}$'), fontsize=20)
ax.set_ylabel('$s_{D}$', fontsize=20)
ax.set_zlabel("$f$", fontsize=20)

# set size
fig.set_size_inches(10, 10)

#for i in range(360):
#    ax.azim = i
#    fig.savefig(f"output/results_regression/image_{i}.png")
#    print(f"saving {i}...")


In [ ]:
# LINEAR REGRESSION OF TRANSFORMED SCORES (2D)

import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

# linear regression
np_all_mapped_scores = np.array(all_mapped_scores)
np_all_metrics = np.array(all_metrics)

m, b = np.polyfit(np_all_mapped_scores, np_all_metrics, 1)

print(m, b)

print(f"metric ={m : .3f} * (depth_score / (depth_score + color_score)) + {b : .3f}")
print(f"R^2 = {r2_score(np_all_metrics, m * np_all_mapped_scores + b)}")

# create plot
fig = plt.figure()
ax = fig.add_subplot()

# scatter points, colored by source image
for (i, cc) in zip(range(3), ['r', 'g', 'b']):
    ax.scatter(mapped_scores[i], metrics[i], color=cc, alpha=0.3)

# plot regression line
plt.plot(np_all_mapped_scores, m * np_all_mapped_scores + b)

# set axes
ax.set_xlabel("$s_{D} / (s_{C} + s_{D})$", fontsize=20)
ax.set_ylabel("$f$", fontsize=20)

# set size
fig.set_size_inches(10, 10)

# save
fig.savefig("output/results_remapped_regression.png")


In [ ]:
# LINEAR REGRESSION OF TRANSFORMED SCORES (3D)

import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

# create plot
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

# scatter points, colored by source image
for (i, cc) in zip(range(3), ['r', 'g', 'b']):
    ax.scatter(color_scores[i], depth_scores[i], metrics[i], color=cc)

# plot regression plane
xx, yy = np.meshgrid(range(11), range(11))
np_xx, np_yy = np.array(xx) * 0.1, np.array(yy) * 0.1

# 0.847 * (depth_score / (depth_score + color_score)) + -0.028
print("metric = 0.847 * (depth_score / (depth_score + color_score)) + -0.028")

z = 0.847 * np_yy / (np_xx + np_yy) - 0.028

ax.plot_surface(np_xx, np_yy, z, alpha=0.2)

# calculate coefficient of determination
np_all_depth_scores = np.array(all_depth_scores)
np_all_color_scores = np.array(all_color_scores)

print(f"R^2 = {r2_score(np_all_metrics, 0.847 * np_all_depth_scores / (np_all_depth_scores + np_all_color_scores) - 0.028)}")

# set axes
ax.set_xlabel(('$s_{C}$'), fontsize=20)
ax.set_ylabel('$s_{D}$', fontsize=20)
ax.set_zlabel("$f$", fontsize=20)

# set size
fig.set_size_inches(10, 10)

#for i in range(360):
#    ax.azim = i
#    fig.savefig(f"output/results_remapped_regression_3d/image_{i}.png")
#    print(f"saving {i}...")
